In [1]:
import os
import sys
import random
import torch
import torchvision
import pytesseract
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import transforms

import cv2
import numpy as np

from utils import (
    overlay_ann,
    overlay_mask,
    show,
    extract_elements
)

In [2]:
seed = 1234
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


CATEGORIES2LABELS = {
    0: "bg",
    1: "text",
    2: "title",
    3: "list",
    4: "table",
    5: "figure"
}

In [3]:
def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )
    return model

In [4]:
def main(argv):
    num_classes = 6
    model = get_instance_segmentation_model(num_classes)
    model.cuda()

    if os.path.exists('model_196000.pth'):
        checkpoint_path = "model_196000.pth"
    else:
        checkpoint_path = "../../../Downloads/model_196000.pth"

    assert os.path.exists(checkpoint_path)
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model.eval()

    # NOTE: custom  image
    if len(argv) > 0 and os.path.exists(argv[0]):
        image_path = argv[0]
    else:
        image_path = 'ieee-research-paper.png'

    assert os.path.exists(image_path)

    image = cv2.imread(image_path)
    rat = 1300 / image.shape[0]
    image = cv2.resize(image, None, fx=rat, fy=rat)

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])
    image = transform(image)

    with torch.no_grad():
        prediction = model([image.cuda()])

    image = torch.squeeze(image, 0).permute(1, 2, 0).mul(255).numpy().astype(np.uint8)
    
    ROI_number = 0 
    for pred in prediction:
        for idx, mask in enumerate(pred['masks']):
            if pred['scores'][idx].item() < 0.7:
                continue

            m = mask[0].mul(255).byte().cpu().numpy()
            box = list(map(int, pred["boxes"][idx].tolist()))
            label = CATEGORIES2LABELS[pred["labels"][idx].item()]

            score = pred["scores"][idx].item()

            # image = overlay_mask(image, m)
            extract_elements(image, box, label, ROI_number)
            # image = overlay_ann(image, m, box, label, score)
            ROI_number += 1
    image_save_path = '../example_images/{}'.format(os.path.basename(image_path))
    cv2.imwrite(image_save_path, image)
    # show(image)
    

In [5]:
if __name__ == "__main__":
    import sys
    argv = sys.argv[1:]
    main(argv)

FDA has been applied so far to averaged dynamical data
from Molecular Dynamics (MD) simulations. However, the
dynamics of the force distribution within proteins or other
macromolecules, ¢.g. in equilibrium, under a constant load,
a load varying in time, or upon binding of another molecule.
can only be characterized by following the changes of the
internal forces in time, which cannot be easily achieved with
the previous implementation of FDA. We here introduce 4
Time-Resolved Force Distribution Analysis (TRFDA) method,
which adds a temporal component to FDA to enable the
analysis of pairwise forces associated with conformational
changes.

Box:
[512, 229, 923, 461]
Label:
text
ROI number:
0
‘Time-Resolved Force Distbution Analysis (TRFDA) 1s
based on the same concept of using pairwise forces as FDA,
but focuses on their evolution during the MD simulation as
well as on the analysis of large molecular systems. To achieve
these goals, it stores in memory pairwise forces from only
one inte